<a href="https://colab.research.google.com/github/azheng456/stocks/blob/main/SPY_gain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np 

In [3]:
#output data
spy = pd.read_csv('SPY.csv')
spy

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-05-17,206.460007,206.800003,204.229996,204.850006,186.340057,114924900
1,2016-05-18,204.440002,206.300003,203.630005,204.910004,186.394577,120062100
2,2016-05-19,204.059998,204.539993,202.779999,204.199997,185.748718,115430500
3,2016-05-20,204.919998,206.100006,204.860001,205.490005,186.922195,104990400
4,2016-05-23,205.509995,205.839996,204.990005,205.210007,186.667496,58682600
...,...,...,...,...,...,...,...
1253,2021-05-10,422.500000,422.739990,417.809998,417.940002,417.940002,81852400
1254,2021-05-11,413.100006,415.269989,410.059998,414.209991,414.209991,116888000
1255,2021-05-12,411.230011,412.589996,404.000000,405.410004,405.410004,134811000
1256,2021-05-13,407.070007,412.350006,407.019989,410.279999,410.279999,106394000


In [15]:
#general info, test to see initial data
pos = 0
neg = 0
for x in range(len(spy["Close"])-2):
    if ((spy["Close"][x+1]-spy["Close"][x])/spy["Close"][x]*100 > 0):
      pos+=1
    else:
      neg+=1
print("In the past 5 years, the SPY had " + str(pos) +
      " positive days and " + str(neg) + " negative days resulting in a "
      + str(round(pos/(pos+neg)*100,1)) + " percent chance of a green day.")

In the past 5 years, the SPY had 706 positive days and 550 negative days resulting in a 56.2 percent chance of a green day.


Let's test 4 SPY  ranges to see the probability of the following day becoming positive: 1 to 2%, 0 to 1%, 0 to -1%, and -1 to -2%

In [16]:
pos = 0
neg = 0
for x in range(len(spy["Close"])-2):
  if (spy["Close"][x+1]>= 1.01*spy["Close"][x] and spy["Close"][x+1]<= 1.02*spy["Close"][x]):
    if ((spy["Close"][x+2]-spy["Close"][x+1])/spy["Close"][x+1]*100 > 0):
      pos+=1
    else:
      neg+=1
print("For days with a SPY gain between 1 and 2%, the next day had " + str(pos) +
      " positive days and " + str(neg) + " negative days in the past 5 years, "+
      " resulting in a "
      + str(round(pos/(pos+neg)*100,1)) + " percent chance of a green day.")


For days with a SPY gain between 1 and 2%, the next day had 70 positive days and 53 negative days in the past 5 years,  resulting in a 56.9 percent chance of a green day.


In [18]:
pos = 0
neg = 0
for x in range(len(spy["Close"])-2):
  if (spy["Close"][x+1]>= spy["Close"][x] and spy["Close"][x+1]<= 1.01*spy["Close"][x]):
    if ((spy["Close"][x+2]-spy["Close"][x+1])/spy["Close"][x+1]*100 > 0):
      pos+=1
    else:
      neg+=1
print("For days with a SPY gain between 0 and 1%, the next day had " + str(pos) +
      " positive days and " + str(neg) + " negative days in the past 5 years,"+
      " resulting in a "
      + str(round(pos/(pos+neg)*100,1)) + " percent chance of a green day.")

For days with a SPY gain between 0 and 1%, the next day had 302 positive days and 257 negative days in the past 5 years, resulting in a 54.0 percent chance of a green day.


In [17]:
pos = 0
neg = 0
for x in range(len(spy["Close"])-2):
  if (spy["Close"][x+1]<= spy["Close"][x] and spy["Close"][x+1]>= 0.99*spy["Close"][x]):
    if ((spy["Close"][x+2]-spy["Close"][x+1])/spy["Close"][x+1]*100 > 0):
      pos+=1
    else:
      neg+=1
print("For days with a SPY loss between 0 and 1%, the next day had " + str(pos) +
      " positive days and " + str(neg) + " negative days in the past 5 years,"+
      " resulting in a "
      + str(round(pos/(pos+neg)*100,1)) + " percent chance of a green day.")

For days with a SPY loss between 0 and 1%, the next day had 259 positive days and 182 negative days in the past 5 years, resulting in a 58.7 percent chance of a green day.


In [19]:
pos = 0
neg = 0
for x in range(len(spy["Close"])-2):
  if (spy["Close"][x+1]<= 0.99*spy["Close"][x] and spy["Close"][x+1]>= 0.98*spy["Close"][x]):
    if ((spy["Close"][x+2]-spy["Close"][x+1])/spy["Close"][x+1]*100 > 0):
      pos+=1
    else:
      neg+=1
print("For days with a SPY loss between 1 and 2%, the next day had " + str(pos) +
      " positive days and " + str(neg) + " negative days in the past 5 years,"+
      " resulting in a "
      + str(round(pos/(pos+neg)*100,1)) + " percent chance of a green day.")


For days with a SPY loss between 1 and 2%, the next day had 32 positive days and 28 negative days in the past 5 years, resulting in a 53.3 percent chance of a green day.


Interestingly enough, the optimal range between these 4 was a loss between 0 and 1%. This led to a 58.7% chance of a following green day. It was then followed by the ranges 1 to 2%, 0 to 1% and -1 to -2% came last. All 4 ranges, however, still had a >50% chance of a green day.

Now we want to examine the entire dataset (so include days with a bigger change than 2%) and break the ranges even smaller to find the optimal predictor of a following positive SPY day.

We will make an array of small increments and iterate through the interval to find the increment that results in the highest percentage chance of a positive day.

In [27]:
percent = 0
ratios = [0, 0.95, 0.96, 0.97, 0.975, 0.98, 0.985, 0.99, 0.995, 1, 1.005, 1.01, 1.015, 1.02, 1.025, 1.03, 1.04, 1.05, 2]
for y in range(len(ratios)-1): #iterate through ratios array
    for x in range(len(spy["Close"])-2): #iterate through SPY data
      #if spy close of next day is between ratios[y] and ratios[y+1]
      if (spy["Close"][x+1]>= ratios[y]*spy["Close"][x] and spy["Close"][x+1]<= ratios[y+1]*spy["Close"][x]):
        if ((spy["Close"][x+2]-spy["Close"][x+1]) > 0):
          pos+=1
        else:
          neg+=1
      if pos/(neg+pos) > percent: #if ratio is max
        percent = pos/(pos+neg)
        r1 = (ratios[y] - 1)*100
        r2 = (ratios[y+1] - 1)*100

print ("The highest percentage of the next " +
      "day being positive was if the previous day had a change of between " + str(round(r1,2)) +
       "% and " + str(round(r2,2)) + "%, with a probability of " + str(round(percent*100,1)) + "%.") 

The highest percentage of the next day being positive was if the previous day had a change of between -0.5% and 0%, with a probability of 56.4%.


Our conclusion finds that with our set ranges and with this 5 year dataset from May 17, 2016 to May 14, 2021, the highest likelihood of a following positive day was if the previous day was between 0 and -0.5%, or a very slight loss. This had a positive ratio of 1.29.